<a href="https://colab.research.google.com/github/Jtandoh01/Disease-Prediction-ML-DCIT311/blob/master/End_of_semester_project_DCIT313.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic Library Imports

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
data = pd .read_csv("./diabetes.csv")
